In [14]:
# Deeper dive into featurization methods for GA vibration data
# We ultimately want something predective . . . where we can see start to move into "scary" land
# We'll compare blunt frequency analysis methods (band pass energy aggregation) and PCA

In [15]:
import pandas as pd
from numpy.fft import fft, fftshift
from scipy.signal import welch
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
#from plotnine import *
from sklearn.cluster import KMeans, AgglomerativeClustering

%matplotlib notebook
import matplotlib
matplotlib.rcParams['figure.figsize'] = [6, 2]
from random import randint

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


%matplotlib notebook
import matplotlib
matplotlib.rcParams['figure.figsize'] = [6, 2]
from random import randint

from matplotlib import animation

In [16]:
def read_ga_file(file_path):
    df = pd.read_csv(file_path, header=None)
    ncol = df.shape[1]
    start_cols = ["id", 'path', "something", "axis", "unix_time", "some_number", "number_samples"] 
    end_cols = range(0, ncol-len(start_cols))
    cols = start_cols + list(end_cols)
    df.columns = cols
    df['datetime'] = pd.to_datetime(df['unix_time'], unit='ms')
    return df

In [17]:
def compute_spectra_from_df(df):
    nums = np.arange(0, 1650)
    S = df[nums].values
    X = np.zeros((S.shape[0], 413))
    for ii in range(S.shape[0]):
        Ft, Xt = welch(S[ii, :], nfft=825)
        X[ii, :] = Xt
    return Ft, X


In [18]:
def compute_spectra_from_df_parseval(df):
    num_samples = 1650
    fs = 4096
    nums = list(range(1650))
    Ft = np.linspace(0, fs/2, num_samples)
    S = df[nums].values
    X = np.zeros((S.shape[0], 1650))
    for ii in range(S.shape[0]):
        Xt = fftshift(fft(S[ii, :]))
        Xt = Xt * np.conj(Xt)
        Xt = np.real(Xt) / len(Xt)
        X[ii, :] = Xt
        #print(len(X))
        #print(np.sum(Xt))
        #print(np.sum(S[ii, :]**2))
        #print()
    return Ft, X

In [21]:
#data_files_path = r"C:\MyData\GP\Waveform"
#results_path = r"C:\MyData\GP\FFT"
#data_files_path = r"D:\Projects\GP QRCM\Waveform Data"
data_files_path = r"P:\Open Projects\Georgia Pacific\Open Projects\3022\Vibration Data\Vibration 7.21"
results_path = r"P:\Open Projects\Georgia Pacific\Open Projects\3022\Vibration Data\fft results"

os.chdir(data_files_path)
current_path = os.getcwd()
print(f"Current Path is set to {current_path}")


for root, dirs, files in os.walk(data_files_path):
    #print(root)
    for file in files:
        file_path = os.path.join(root, file)
        #print(f"Root is {root}")
        full_results_path = root.replace(data_files_path, results_path)
        #print(f"FFT Results path is {full_results_path}")
        os.makedirs(full_results_path, exist_ok = True)
        
        # Get file extension
        #print(os.path.splitext(file_path)[1])
        
        
        print(f"\t{file_path}")
        
        output_file_path = os.path.join(full_results_path, file)
            
        df = read_ga_file(file_path)
        print(df.shape)
        
        Ft, X = compute_spectra_from_df_parseval(df)
        freq_dft = pd.DataFrame(X)
        freq_dft["axis"] = df["axis"].values
        freq_dft["path"] = df["path"].values
        freq_dft["datetime"] = df["datetime"].values

        # Need this to translate integer dataframe columns to frequencies while not messing up the non-integer ones
        in_cols = freq_dft.columns.values
        out_cols = []
        for col in in_cols:
            if col in ['axis', 'datetime', 'date', 'path']:
                out_cols.append(col)
            else:
                out_cols.append(4096*col/824)

        conv_dict = dict(zip(in_cols, out_cols))

        freq_dft = freq_dft.rename(columns=conv_dict)
        freq_dft["date"] = freq_dft["datetime"].apply(lambda x: x.date())
        freq_dft.to_csv(output_file_path, index=None)
            
            

Current Path is set to P:\Open Projects\Georgia Pacific\Open Projects\3022\Vibration Data\Vibration 7.21
	P:\Open Projects\Georgia Pacific\Open Projects\3022\Vibration Data\Vibration 7.21\FD-0503 #13 Washer Reducer f07b35d3-52e2-4280-a9bc- 7286fabddcfe 1\GIB 057e3bc8-6515-46dd-8666-5ca4e71c52f4\00008227\X\0122.csv
(8140, 1658)


KeyboardInterrupt: 